In [1]:
!pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd


In [3]:
file_path = r"C:\my revision\bank statement 1.xlsx"  # 👈 change this path
all_sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

print("Number of sheets found:", len(all_sheets))


Number of sheets found: 63


In [4]:
# Get standard header from the first sheet
first_sheet_name = list(all_sheets.keys())[0]
standard_columns = all_sheets[first_sheet_name].columns
print("Header columns:", list(standard_columns))


Header columns: ['Trans. Time', 'Value Date', 'Description', 'Debit/Credit(₦)', 'Balance(₦)', 'Channel', 'Transaction Reference', 'Counterparty']


In [6]:
combined_df = pd.DataFrame()

for name, sheet in all_sheets.items():
    print(f"Processing {name}...")

    # Drop completely blank rows and columns
    sheet = sheet.dropna(how='all').dropna(axis=1, how='all')

    # Make sure number of columns match
    if len(sheet.columns) != len(standard_columns):
        sheet = sheet.iloc[:, :len(standard_columns)]  # keep only first N columns

    # Assign consistent headers
    sheet.columns = standard_columns

    # Drop extra blank rows
    sheet = sheet.dropna(how='all')

    # Combine
    combined_df = pd.concat([combined_df, sheet], ignore_index=True)

print("✅ All sheets combined successfully!")


Processing Table 2...
Processing Table 3...
Processing Table 4...
Processing Table 5...
Processing Table 6...
Processing Table 7...
Processing Table 8...
Processing Table 9...
Processing Table 10...
Processing Table 11...
Processing Table 12...
Processing Table 13...
Processing Table 14...
Processing Table 15...
Processing Table 16...
Processing Table 17...
Processing Table 18...
Processing Table 19...
Processing Table 20...
Processing Table 21...
Processing Table 22...
Processing Table 23...
Processing Table 24...
Processing Table 25...
Processing Table 26...
Processing Table 27...
Processing Table 28...
Processing Table 29...
Processing Table 30...
Processing Table 31...
Processing Table 32...
Processing Table 33...
Processing Table 34...
Processing Table 35...
Processing Table 36...
Processing Table 37...
Processing Table 38...
Processing Table 39...
Processing Table 40...
Processing Table 41...
Processing Table 42...
Processing Table 43...
Processing Table 44...
Processing Table 45

In [7]:
combined_df.to_excel("combined_bank_statement_cleaned.xlsx", index=False)


In [9]:
import pandas as pd

# Step 1 — Load all sheets
file_path = r"C:\Users\THIS-PC\Jupyter work\combined_bank_statement_cleaned.xlsx"  # 👈 change this to your actual path
all_sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Step 2 — Define your standard columns manually
standard_columns = [
    'Trans. Time',
    'Value Date',
    'Description',
    'Debit/Credit(₦)',
    'Balance(₦)',
    'Channel',
    'Transaction Reference',
    'Counterparty'
]

# Step 3 — Combine safely
combined_df = pd.DataFrame()

for name, sheet in all_sheets.items():
    print(f"Processing sheet: {name}")

    # Drop completely blank rows/columns
    sheet = sheet.dropna(how='all').dropna(axis=1, how='all')

    # Keep only the first 8 columns if more exist
    if len(sheet.columns) > len(standard_columns):
        sheet = sheet.iloc[:, :len(standard_columns)]

    # If there are fewer columns, pad them with empty ones
    elif len(sheet.columns) < len(standard_columns):
        for i in range(len(sheet.columns), len(standard_columns)):
            sheet[f"Missing_{i}"] = None

    # Assign consistent column names
    sheet.columns = standard_columns

    # Drop fully empty rows
    sheet = sheet.dropna(how='all')

    # Combine
    combined_df = pd.concat([combined_df, sheet], ignore_index=True)

print("✅ All sheets combined successfully!")

# Step 4 — Final clean-up
combined_df = combined_df.drop_duplicates().reset_index(drop=True)

# Step 5 — Save to Excel
combined_df.to_excel("combined_bank_statement_cleaned.xlsx", index=False)

print("✅ Saved as 'combined_bank_statement_cleaned.xlsx'")


Processing sheet: Sheet1
✅ All sheets combined successfully!
✅ Saved as 'combined_bank_statement_cleaned.xlsx'


In [10]:
import pandas as pd

# Load your cleaned combined file
file_path = r"C:\Users\THIS-PC\Jupyter work\combined_bank_statement_cleaned.xlsx"  # 👈 update path
df = pd.read_excel(file_path)

df.head()


,Trans. Time,Value Date,Description,Debit/Credit(₦),Balance(₦),Channel,Transaction Reference,Counterparty
0,2024 Jun 30 05:24:32,2024-06-30 00:00:00,Transfer to UDE BLESSING,-35000,551616.32,E-Channel,240630023694173638,First Bank Of Nigeria | 3087300146
1,2024 Jun 30 05:25:30,2024-06-30 00:00:00,Transfer to AZUTURU OKORO OKORO,-5000,546616.32,E-Channel,240630023694224750,United Bank For Africa | 2147093132
2,2024 Jun 30 15:04:18,2024-06-30 00:00:00,Transfer to Gods grace pos 2,-6850,539766.32,E-Channel,240630024088044842,MONIE POINT | 8273964299
3,2024 Jun 30 15:07:31,2024-06-30 00:00:00,Transfer to Gods grace pos 2,-4310,535456.32,E-Channel,240630024078518301,MONIE POINT | 8273964299
4,2024 Jun 30 18:53:43,2024-06-30 00:00:00,Transfer to SHARWAMA DELIGHT LOM ALINDA,-3010,532446.32,E-Channel,240630024290666559,MONIE POINT | 5432873681


In [11]:
print(df.columns.tolist())


['Trans. Time', 'Value Date', 'Description', 'Debit/Credit(₦)', 'Balance(₦)', 'Channel', 'Transaction Reference', 'Counterparty']


In [12]:
# Create new Transaction Type and Amount columns
df['Transaction Type'] = df['Debit/Credit(₦)'].apply(
    lambda x: 'Debit' if str(x).startswith('-') or '(' in str(x) else 'Credit'
)

# Convert Amount to numeric (remove ₦, commas, brackets, etc.)
df['Amount'] = df['Debit/Credit(₦)'].astype(str)
df['Amount'] = df['Amount'].str.replace('[₦,()]', '', regex=True).str.strip()
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Convert Debit values to negative
df.loc[df['Transaction Type'] == 'Debit', 'Amount'] *= -1


In [13]:
# Convert date columns
df['Value Date'] = pd.to_datetime(df['Value Date'], errors='coerce')

# Clean Balance column
df['Balance(₦)'] = df['Balance(₦)'].astype(str).str.replace('[₦,()]', '', regex=True).str.strip()
df['Balance(₦)'] = pd.to_numeric(df['Balance(₦)'], errors='coerce')


In [14]:
df = df[['Trans. Time', 'Value Date', 'Description', 'Transaction Type', 'Amount',
         'Balance(₦)', 'Channel', 'Transaction Reference', 'Counterparty']]


In [16]:
df.head(50)

,Trans. Time,Value Date,Description,Transaction Type,Amount,Balance(₦),Channel,Transaction Reference,Counterparty
0,2024 Jun 30 05:24:32,2024-06-30,Transfer to UDE BLESSING,Debit,35000.0,551616.32,E-Channel,240630023694173638,First Bank Of Nigeria | 3087300146
1,2024 Jun 30 05:25:30,2024-06-30,Transfer to AZUTURU OKORO OKORO,Debit,5000.0,546616.32,E-Channel,240630023694224750,United Bank For Africa | 2147093132
2,2024 Jun 30 15:04:18,2024-06-30,Transfer to Gods grace pos 2,Debit,6850.0,539766.32,E-Channel,240630024088044842,MONIE POINT | 8273964299
3,2024 Jun 30 15:07:31,2024-06-30,Transfer to Gods grace pos 2,Debit,4310.0,535456.32,E-Channel,240630024078518301,MONIE POINT | 8273964299
4,2024 Jun 30 18:53:43,2024-06-30,Transfer to SHARWAMA DELIGHT LOM ALINDA,Debit,3010.0,532446.32,E-Channel,240630024290666559,MONIE POINT | 5432873681
5,2024 Jul 01 08:30:58,2024-07-01,Transfer to CHINELO LILIAN ASADU,Debit,55000.0,477446.32,E-Channel,240701014621348264,OPay | 9035351128
6,2024 Jul 02 02:50:18,2024-07-02,Transfer from BUD INFRASTRUCTURE LTD,Credit,115717.0,593163.32,E-Channel,000030240702035017361120\n041930,Parallex Bank | 1000130886
7,2024 Jul 02 03:00:05,2024-07-02,Transfer from Transfer,Credit,59448.0,652611.32,E-Channel,0904052407020400053792\n08252843,MONIE POINT | 1017432705
8,2024 Jul 02 03:42:03,2024-07-02,Transfer from Transfer,Credit,235794.0,888405.32,E-Channel,0904052407020442034508\n59176970,MONIE POINT | 1017432705
9,2024 Jul 02 08:44:35,2024-07-02,Transfer to CHUKWUEMEKA DESMOND NNAMANI,Debit,820000.0,68405.32,E-Channel,240702025522109831,United Bank For Africa | 2308056659


In [17]:
df = df.dropna(how='all')
print("✅ Fully empty rows removed.")


✅ Fully empty rows removed.


In [18]:
df = df.dropna(subset=['Description', 'Amount'], how='any')
print("✅ Removed rows without Description or Amount.")


✅ Removed rows without Description or Amount.


In [20]:
df['Trans. Time'] = df['Trans. Time'].fillna('00:00')


In [22]:
df = df.reset_index(drop=True)
df.to_excel("bank_statement_powerbi_ready_clean.xlsx", index=False)

print("✅ Clean Power BI file saved successfully!")


✅ Clean Power BI file saved successfully!
